In [56]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import tarfile
import glob
from matplotlib.colors import LogNorm
import pathlib
import pickle

In [73]:
# Inputs ETH
run_type = 'ov'

if run_type == 'brca':
   
    sample_target = 'TCGA-AO-A0JM-01A-21R-A056-07'
    samples = ['TCGA-C8-A12P-01A-11R-A115-07',
                      'TCGA-AO-A0JM-01A-21R-A056-07',
                      'TCGA-BH-A18V-01A-11R-A12D-07',
                      'TCGA-A2-A0D2-01A-21R-A034-07',
                      'TCGA-A2-A0SX-01A-12R-A084-07']
    basedir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102'
elif run_type == 'ov':
    sample_target = 'TCGA-25-1319-01A-01R-1565-13'
    samples = ['TCGA-25-1319-01A-01R-1565-13',
                      'TCGA-25-1313-01A-01R-1565-13',
                      'TCGA-61-2008-01A-02R-1568-13',
                      'TCGA-24-1431-01A-01R-1566-13',
                      'TCGA-24-2298-01A-01R-1569-13']
    basedir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374'

filtering_id = 'filters_19May_order_5ge_wAnnot_GPstar'
filter_dir = os.path.join(basedir, 'filtering_samples', filtering_id)


# PLOT
plot_dir = os.path.join(filter_dir, 'plots')
base_plot = sample_target
pathlib.Path(plot_dir).mkdir(parents=True, exist_ok=True)

# OHSU
files_OHSU = '/cluster/work/grlab/projects/projects2020_OHSU/share_OHUS_PNLL/current/kmer_files'

In [74]:
# ETH Names
eth_all = glob.glob(os.path.join(filter_dir, 'G*'))
eth_all = [i for i in eth_all if len(os.path.basename(i)) == 45] #Shorten path has len 45


In [75]:
# OHSU Names
ohsu_all = glob.glob(os.path.join(files_OHSU, 'J*'))

In [76]:
# Get file pairs
file_pair = {}
for idx_eth, eth in enumerate(eth_all):
    eth_pattern = os.path.basename(eth).replace('G_', '').replace('.gz', '') 
    for idx_ohsu, ohsu in enumerate(ohsu_all):
        if eth_pattern in ohsu:
            file_pair[eth] = ohsu

            

In [77]:
print(len(file_pair))

175


In [78]:
all_ohsu = set(ohsu_all) #TODO PLOT THE OHSU motifs matching ETH
print(len(all_ohsu))

1800


In [79]:
found_ohsu = set(file_pair.values())
found_eth = set(file_pair.keys()) 

In [80]:
all_eth = set(eth_all)
print(len(all_eth))

325


In [81]:
len(all_eth - found_eth)

150

In [82]:
len(all_ohsu - found_ohsu)

1625

In [83]:
print(len(all_ohsu & all_eth))

0


# Collect Data to plot

In [84]:
### Compute the intersection sizes on the fly for the sample
restrict = ''
df = {'sample' : [], 
      'filter_' : [], 
      #'filter_foreground' : [], 
      #'filter_background' : [], 
      'size_ohsu' : [], 
      'size_eth' : [], 
      'size_intersection' : [], 
      'size_ohsu\eth' : [], 
      'size_eth\ohsu' : []}
for eth, ohsu in file_pair.items(): # ETH
    if (not restrict) or restrict in eth: #Restrict to category of interest
        try:
            df_ohsu = pd.read_csv(ohsu, sep="\t")
            df_ohsu = df_ohsu.reset_index().rename({'index': 'kmer', 'kmer': 'coord'}, axis = 1) 
            df_eth = pd.read_csv(eth, sep="\t", usecols = ['kmer'])

            df_eth = set(df_eth['kmer'])
            df_ohsu = set(df_ohsu['kmer'])
            
            name = os.path.basename(eth).replace('.tsv', '').replace('.gz', '').split('_')
            
            df['sample'].append(name[1])
            df['filter_'].append(name[2])
            df['size_ohsu'].append(len(df_ohsu))
            df['size_eth'].append(len(df_eth))
            df['size_ohsu\eth'].append(len(df_ohsu.difference(df_eth)))
            df['size_eth\ohsu'].append(len(df_eth.difference(df_ohsu)))
            df['size_intersection'].append(len(df_ohsu.intersection(df_eth)))
            #                 df['filter_foreground'].append(name[2])
#                 df['filter_background'].append(name[3])
        except:
            continue


In [85]:
df = pd.DataFrame(df)

In [86]:
save_folder = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/data_plots'

In [87]:
file_name = f'data_kmers_filtering_ohsu_eth_{run_type}.tsv.gz'

In [88]:
print(os.path.join(save_folder, file_name))
df.to_csv(os.path.join(save_folder, file_name), index=None, sep='\t')

/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/data_plots/data_kmers_filtering_ohsu_eth_ov.tsv.gz
